# Covid Tweets Dataset Cleaning

#### This script is used to prepare and clean a small, example sample of the Covid Tweets Dataset. The main idea of this script is to demonstrate some of the cleaning capabilities of the framework. For the experiment a sample of 1000 (rows) in English will be used. The dataset contains 9 errors types previously added from the "Insert Dirty Data" script and also contains human-generated text from the posts that will be prepared and cleaned through NLP (Natural Language Processing) tasks. The first section of the script is the NLP tasks, which are as follows: 
    1. Removing emojis. 
    2. Removing hyperlinks.
    3. Removing new lines (\n).
    4. Removing HTML entities. 
    5. Removing hashtags (#) and tagged accounts (@).
    6. Splitting the attached strings.
    7. Lowercasing all words.
    8. Transforming contractions.
    9. Removing punctuations.
    10. Transforming slang words.
    11. Removing stop words.
#### The second section of the script is cleaning the errors types in the other columns, which are as follows:
    1. Cleaning wrong data type errors. 
    2. Cleaning negative number representation errors.
    3. Cleaning extraneous data.
    4. Swapping row ordering.
    5. Placing data into correct fields.
    6. Cleaning duplicated data.
    7. Cleaning misspelled data.
    8. Cleaning special character errors.
    9. Cleaning inconsistent data.

#### Installations

In [28]:
# For output cleaning
from IPython.display import clear_output

# Remove the comments in order to perform the necessary installations!

#!pip install opencv-python
#!pip install numpy
#!pip install pandas
#!pip install nltk
#!pip install re
#!pip install xlrd
#!pip install openpyxl

# Clear the output
clear_output()

#### Imports

In [5]:
# Import the necessary libraries.
import re 
import nltk
import math
import pandas as pd
from nltk.corpus import stopwords
from difflib import SequenceMatcher

# Clear the output
clear_output()

### 1. NLP processing of text data. 

Download all stopwords  

In [49]:
# Download stopwords
nltk.download('stopwords')

# Change dataframe column and row width limits
pd.options.display.max_colwidth = 10000
pd.set_option('display.max_rows', None)

# Clear the output
clear_output()

Load 1651 tweets into a dataframe. 

In [160]:
# Exactly 1651 tweets are being loaded because 651 are not in English, and later in the code will be removed.
# Leaving 1000 rows for testing.
df = pd.read_csv('D:\TweetCSV\TweetsWorkbookCSV.csv', nrows=1651)

Remove all unnecessary columns.

In [161]:
# Removes all other columns and focuses only on the "text" column.
df.drop(['coordinates', 'created_at', 'hashtags', 'media', 'urls', 'in_reply_to_status_id', 'in_reply_to_user_id', 
         'user_followers_count', 'favorite_count', 'in_reply_to_screen_name', 'place', 'possibly_sensitive', 
         'quote_id', 'retweet_count', 'retweet_id', 'retweet_screen_name', 'user_created_at', 'user_id', 
         'user_default_profile_image', 'user_description', 'user_favourites_count', 'user_screen_name',
         'user_statuses_count', 'user_time_zone', 'user_urls', 'user_verified', 'source', 'tweet_url', 
         'user_friends_count', 'user_listed_count', 'user_location', 'user_name'], inplace=True, axis=1)

Remove all rows that are not in English. Leaving exactly 1000 rows in English.

In [162]:
# Leaves only the rows that are in english language.
df.drop(df.index[df['lang'] != 'en'], inplace=True)

#### Remove all emojis. 

In [163]:
# Remove all emojis through the use of a lambda.
filter_char = lambda c: ord(c) < 256
df['text'] = df['text'].apply(lambda s: ''.join(filter(filter_char, s)))

#### Remove all hyperlinks.

In [164]:
# Remove hyperlinks through a regex.
df = df.replace('http[s]?:\S+|www.\S+', '',regex=True)

#### Remove all new lines (\n).

In [165]:
# Remove all new lines and replace them with empty space.
df['text'] = df['text'].str.replace('\n', ' ')

#### Remove all HTML entities such as &apos, &amp and so on.

In [166]:
# Remove all HTML.
df = df.replace('&\S+', '',regex=True)

#### Remove all tagged accounts and all hashtag words.

In [167]:
# Remove all tagged accounts and remove all # signs, leaving the words. 
df = df.replace('@\S+', '',regex=True)
df = df.replace('#', '',regex=True)

#### Split all attached strings.

In [168]:
# Split all attached strings that could have been left from the hashtags or other places.
for key, value in df['text'].iteritems():
    df.loc[key, 'text'] = " ".join([s for s in re.split("([A-Z][a-z]+[^A-Z]*)", value) if s])

#### Turn all words to lowercase.

In [169]:
# Turn all words to lowercase through a lambda.
df['text'] = df['text'].apply(lambda x:x.lower())

#### Transform all contractions to the actual words. Such as "can't" = "cannot"

In [170]:
# Open the text file with all contractions.
file=open("D:\Contractions.txt","r")
contractions=file.read()
 
# Separate each line from the file.
contractions=contractions.split('\n')
 
contr=[]
contr_meaning=[]
 
# Store the contractions and their actual meanings in different lists.
for line in contractions:
    temp=line.split(":")
    contr.append(temp[0])
    contr_meaning.append(temp[-1])

# Loop through all rows in the dataframe.
for row in df['text'].iteritems():
    wordList = row[1].split(" ")
    # Loop through all words in a row.
    for word in wordList:
        # Loop through all contractions.
        for c in enumerate(contr):
            # Compare each word with each contraction.
            if word == c[1]:
                index = wordList.index(word)
                idx=c[0]
                # Swap contraction with its meaning.
                wordList[index] = contr_meaning[idx]
        # Swap old row with new one in the dataframe. 
        df.loc[row[0], 'text'] = " ".join(wordList)

#### Remove all punctuations.

In [171]:
# Remove all punctuations through the use of a regex.
df = df.replace('[^\w\s]', '',regex=True)

Turn the text into arrays. For better processing of stop words and slang words.

In [172]:
# Create a new DF object to temporarily store transformed rows.
temp_df = pd.DataFrame(columns=['TextArrays'])

# Loop through all rows in the dataframe.
for key, value in df['text'].iteritems():
    temp_list = value.split()
    # Remove all empty or spaced objects in the list.
    if '' in temp_list:
        temp_list.remove('')
    if ' ' in temp_list:
        temp_list.remove(' ')
    # Append the lists to the temporary dataframe.
    temp_df = temp_df.append({'TextArrays': temp_list}, ignore_index=True)

#### Turning slang words to actual words.

In [173]:
# Open the text file with all slang words.
file=open("D:\SlangWordsMeaning.txt","r")
slangWords=file.read()
 
# Separate each line from the file.
slangWords=slangWords.split('\n')
 
slang_word=[]
meaning=[]
 
# Store the slang words and their actual meanings in different lists.
for line in slangWords:
    temp=line.split(":")
    slang_word.append(temp[0])
    meaning.append(temp[-1])

# Create a new DF object to store transformed rows.
temp_df5 = pd.DataFrame(columns=['Final'])

# Loop through all rows in the dataframe.
for row in temp_df['TextArrays'].iteritems():
    # Loop through all words in a row.
    for word in row[1]:
        # Loop through all slang words.
        for slang in enumerate(slang_word):
            # Compare each word with each slang word.
            if word == slang[1]:
                index = row[1].index(word)
                idx=slang[0]
                # Remove previous slang word from list.
                del row[1][index]
                # Append the meaning of the slang word as different list objects.
                for x in meaning[idx].split():
                  row[1].append(x)
    # Add the transformed rows to the new dataframe.
    temp_df5 = temp_df5.append({'Final': row[1]}, ignore_index=True)

#### Remove stopwords.

In [174]:
# Create a new DF object to store transformed rows.
temp_df2 = pd.DataFrame(columns=['Final'])

# Loop through all rows in the dataframe.
for key, value in temp_df5['Final'].iteritems():
    # Iter a copy of each row (used to prevent bugs in removal).
    for word in list(value):
        # If a word is a stop word, remove it.
        if word in stopwords.words('english'):
            value.remove(word)
    # Add the transformed rows to the new dataframe.    
    temp_df2 = temp_df2.append({'Final': value}, ignore_index=True)

Save the arrays of words to an excel file.

In [175]:
# Save
temp_df2.to_excel("D:\Text_After_NLP.xlsx")

### 2. Cleaning dirty data from the Tweets Dataset 

Read the file.

In [148]:
# Read the dirty file
df2 = pd.read_excel("D:\DirtyTweetsDataset.xlsx", engine='openpyxl')

Remove redundant columns that have been created when saving.

In [149]:
# Remove unnecessary columns 'user_default_profile_image'] (if there are any)
df2.drop(['Unnamed: 0', 'Unnamed: 0.1'], inplace=True, axis=1)

#### Cleaning the inconsistent data in the 'lang' column.

In [150]:
empt_dict = {}

# Through the use of a dictionary get each row and its number of occurances 
for index, row in df2.iterrows():
    l = row['lang']
    if l in empt_dict:
        empt_dict[l] += 1
    else: 
        empt_dict[l] = 1

# Get the most frequent row 
most_frequent_key = max(empt_dict, key=empt_dict.get)

# Update all rows to be the same as the most frequent
for index, row in df2.iterrows():
    if row['lang'] == most_frequent_key:
        continue
    else:
        df2.loc[index, 'lang'] = most_frequent_key

#### Cleaning the wrong data type errors in the 'user_default_profile_image' column.

In [151]:
empt_dict2 = {}

# Function to get similarity between elements
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()


# Through the use of a dictionary get type of row and its number of occurances 
for index, row in df2.iterrows():
    final = ""
    # First find the type of the row
    # String
    if type(row['user_default_profile_image']) is str:
        final = str
    # Integer 
    elif type(row['user_default_profile_image']) is int:
        final = int
    # Boolean
    elif type(row['user_default_profile_image']) is bool:
        final = bool
    
    # Increment accordingly 
    if final in empt_dict2:
        empt_dict2[final] += 1
    else: 
        empt_dict2[final] = 1

# Get the most frequent type
most_frequent_key2 = max(empt_dict2, key=empt_dict2.get)

# Update the rows accordingly 
for index, row in df2.iterrows():
    if type(row['user_default_profile_image']) == most_frequent_key2:
        continue
    else:
        # Through the use of similarity decide whether to update to true or false
        if similar(row['user_default_profile_image'], 'true') > similar(row['user_default_profile_image'], 'false'):
            df2.loc[index, 'user_default_profile_image'] = True
        else:
            df2.loc[index, 'user_default_profile_image'] = False

#### Cleaning the special characters in the 'user_favourites_count' column.

In [152]:
# Loop through all rows
for index, row in df2.iterrows():
    # Check if row is string (since integer rows can't have special numbers)
    if type(row['user_favourites_count']) == str:
        # Check if row has special characters
        if not row['user_favourites_count'].isalnum():
            # Update row by removing the special character and casting it to int
           df2.loc[index, 'user_favourites_count'] = int(''.join(s for s in row['user_favourites_count'] if s.isalnum()))

#### Cleaning the negative number representation in the 'retweet_count' column.

In [153]:
# Loop through all rows
for index, row in df2.iterrows():
    # Check if number is negative
    if row['retweet_count'] < 0:
        # If num is -1 then it was converted from 0
        if row['retweet_count'] == -1:
            df2.loc[index, 'retweet_count'] = 0
        # Make positive
        else: 
             df2.loc[index, 'retweet_count'] = abs(row['retweet_count'])
    else:
        continue

#### Cleaning the extraoneous data in the 'user_created_at' column.

In [154]:
list_dates = []

# Assume that the extra value is at the begining of the row and is from the user_id since there are blank rows
# Loop through the rows
for index, row in df2.iterrows():
    arr = row['user_created_at'].split(' ')
    # If arr is longer than usual 
    if len(arr) > 6:
        list_dates.append(float(arr[0]))
        arr.pop(0)
        df2.loc[index, 'user_created_at'] =  ' '.join(arr)

# Loop to bring back the id's to their column
for index, row in df2.iterrows():
    # If row is NaN
    if math.isnan(row['user_id']):
        # If list not empty
        if list_dates:
            # Assign and remove
            df2.loc[index, 'user_id'] = list_dates[0]
            del list_dates[0]

#### Cleaning the misspelled data in the 'user_created_at' column.

In [155]:
# Loop through all rows
for index, row in df2.iterrows():
    arr = row['user_created_at'].split(' ')
    empt_dict = {}
    
    # Get the similarities 
    empt_dict['Mon'] = similar(arr[0], 'Mon')
    empt_dict['Tue'] = similar(arr[0], 'Tue')
    empt_dict['Wed'] = similar(arr[0], 'Wed')
    empt_dict['Thu'] = similar(arr[0], 'Thu')
    empt_dict['Fri'] = similar(arr[0], 'Fri')
    empt_dict['Sat'] = similar(arr[0], 'Sat')
    empt_dict['Sun'] = similar(arr[0], 'Sun')
    
    # Get the most similar day of the week
    most_similar_key = max(empt_dict, key=empt_dict.get)
    
    # Assign the new day of the week
    arr[0] = most_similar_key
    df2.loc[index, 'user_created_at'] =  ' '.join(arr)

#### Cleaning the wrongly inserted data in the 'source' and 'created_at' columns.

In [156]:
empt_dict = {}

# Loop through all rows
# Assume that the odd rows must go to the 'source' column 
for index, row in df2.iterrows():
    r = row['source']
    if r in empt_dict:
        empt_dict[r] += 1
    else: 
        empt_dict[r] = 1
    
# Get the most frequent row 
most_frequent_key = max(empt_dict, key=empt_dict.get)

for index, row in df2.iterrows():
    sim = similar(most_frequent_key, row['source'])
    f = float("{:.2f}".format(sim))
    if f < 0.30:
        temp = df2.loc[index, 'source']
        df2.loc[index, 'source'] = df2.loc[index, 'created_at']
        df2.loc[index, 'created_at'] = temp

#### Cleaning the duplicate data in the 'user_id' column.

In [157]:
# It has been decided that the framework will deal with duplicates by eliminating them. 
# Pandas provides a drop_duplicates function which will remove all of the duplicates in the dataframe column.
# 'keep' will leave the first occurance of the 'user_id'.
df2.drop_duplicates(subset ="user_id",
                     keep = 'first', inplace = True)

#### Fixing the swapped orderings in the 'created_at' column.

In [158]:
# Loop through all rows
for index, row in df2.iterrows():
    # Check if row starts with a String, if not then assume its dirty
    if re.search("^[a-zA-Z]", row['created_at']) is not None:
        continue
    # If dirty then clean it
    else:
        arr = row['created_at'].split(' ')
        # Loop through all elements of the row
        for a in arr:
            if re.search("^[a-zA-Z]", a) is not None:
                break
            # Append to the end of the array, to fix order
            else:
                arr.append(a)                
                
        # Delete the redundant elements
        if len(arr) > 6:
            del arr[0:3]
                  
        # Update the rows
        df2.loc[index, 'created_at'] =  ' '.join(arr)

Save the cleaned tweets to an excel file.

In [159]:
# Save
df2.to_excel("D:\Cleaned_English_Tweets.xlsx")